In [1]:
#TODO : REFAIRE FONCTION COMPUTE_MU_SPATIAL POUR ACCELERER

# Importing the packages and data

In [2]:
%load_ext autoreload
%autoreload 2

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import gamma, digamma, polygamma
import math

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import random

import scipy.optimize
from scipy.optimize import Bounds

In [4]:
import sys
sys.path.append("../coral_mapping/")
import segments_regressor
import coral_map_tests
import maupiti_func

In [5]:
import dirichlet_regression

In [6]:
from scipy.optimize import fmin, newton, minimize

In [7]:
img_B02_raw = np.load('../maupiti/Satellite Images/20200922_img_B02.npy')
img_B03_raw = np.load('../maupiti/Satellite Images/20200922_img_B03.npy')
img_B04_raw = np.load('../maupiti/Satellite Images/20200922_img_B04.npy')
img_B08_raw = np.load('../maupiti/Satellite Images/20200922_img_B08.npy')

In [8]:
path_maupiti_zones="../../Maupiti/img_maupiti_zones"
file = open(path_maupiti_zones, "rb")
maupiti_zones = np.load(file)
file.close
maupiti_zones_normalized=maupiti_func.create_zones_normalized(np.array(maupiti_zones, dtype='int'))

In [9]:
map_test=coral_map_tests.coral_map([img_B02_raw,img_B03_raw,img_B04_raw,img_B08_raw], map_img=maupiti_zones_normalized)

In [10]:
map_test.segmentation(method='fz',scale=0.5, min_size=40)
np.max(map_test.segments)

4425

In [11]:
boundaries = coral_map_tests.find_boundaries(map_test.segments)
adjacent_segments,_ = segments_regressor.find_adjacent_segments(map_test.segments, boundaries)

In [12]:
X_maupiti = segments_regressor.create_features(map_test.segments, map_test.blue_band, map_test.green_band, map_test.red_band, map_test.nir_band)

In [13]:
Y_maupiti = coral_map_tests.create_labels_dataset(map_test.segments,map_test.map_img)

# With Maupiti

In [91]:
%%time
test_1 = np.linalg.inv(np.identity(n_samples) - rho_maupiti*W_maupiti)

Wall time: 2.15 s


In [101]:
%%time
test_2 = scipy.sparse.linalg.inv(scipy.sparse.csc_matrix(np.identity(n_samples) - rho_maupiti*W_maupiti))
test_2 = test_2.toarray()

Wall time: 1.26 s


In [102]:
%%time
test_3 = scipy.sparse.linalg.splu(scipy.sparse.csc_matrix(np.identity(n_samples) - rho_maupiti*W_maupiti)).solve(np.eye(n_samples))
test_3 = np.transpose(test_3)

Wall time: 672 ms


In [107]:
def f_spatial(x, X, Y, Z, W, epsilon=0):
    K = X.shape[-1]
    J = Y.shape[-1]
    n = X.shape[0]
    beta = x[:K*J].reshape((K,J))
    beta[:,0] = 0
    gamma_var = x[K*J:-1]
    rho = x[-1]
    #Minv = np.linalg.inv(np.identity(n) - rho*W)
    Minv = scipy.sparse.linalg.splu(scipy.sparse.csc_matrix(np.identity(n) - rho*W)).solve(np.eye(n))
    Minv = np.transpose(Minv)
    mu = dirichlet_regression.compute_mu_spatial(X, beta, rho=rho, W=W, Minv=Minv)
    phi = np.exp(np.matmul(Z,gamma_var))
    return -dirichlet_regression.dirichlet_loglikelihood(mu,phi,Y,epsilon=epsilon)

In [108]:
def fprime_spatial(x, X, Y, Z, W, epsilon=0):
    K = X.shape[-1]
    J = Y.shape[-1]
    n = X.shape[0]
    beta = x[:K*J].reshape((K,J))
    beta[:,0] = 0
    gamma_var = x[K*J:-1]
    rho = x[-1]
    #Minv = np.linalg.inv(np.identity(n) - rho*W)
    Minv = scipy.sparse.linalg.splu(scipy.sparse.csc_matrix(np.identity(n) - rho*W)).solve(np.eye(n))
    Minv = np.transpose(Minv)
    MinvX = np.matmul(Minv,X)
    mu = dirichlet_regression.compute_mu_spatial(X, beta, rho=rho, W=W, MinvX=MinvX)
    phi = np.exp(np.matmul(Z,gamma_var))

    beta_grad = dirichlet_regression.dirichlet_gradient_wrt_beta(mu, phi, MinvX, Y, epsilon=epsilon)
    beta_grad[:,0] = 0
    gamma_grad = dirichlet_regression.dirichlet_derivative_wrt_gamma(mu, phi, beta, MinvX, Y, Z, epsilon=epsilon)
    rho_derivative = dirichlet_regression.dirichlet_derivative_wrt_rho(mu, phi, Minv, beta, W, X, Y, Z, MinvX, epsilon=epsilon)
    return(-np.concatenate([beta_grad.flatten(),gamma_grad,[rho_derivative]]))

In [8]:
#X_maupiti = pd.read_csv('../../Maupiti/maupiti_X_13244.csv', index_col=0)
#Y_maupiti = pd.read_csv('../../Maupiti/maupiti_Y_13244.csv',index_col=0)

In [16]:
n_features = X_maupiti.shape[1]
n_classes = Y_maupiti.shape[1]
n_samples = X_maupiti.shape[0]

In [17]:
X_maupiti = np.array(X_maupiti)
Y_maupiti = np.array(Y_maupiti)

In [18]:
X_maupiti = (X_maupiti-np.min(X_maupiti))/(np.max(X_maupiti)-np.min(X_maupiti))

In [19]:
Z_maupiti = np.ones((n_samples,n_features))

In [20]:
W_maupiti = np.zeros((n_samples,n_samples))


In [21]:
W_maupiti = np.zeros((n_samples,n_samples))
for i in range(n_samples):
    W_maupiti[i,adjacent_segments[i]]=1

In [22]:
W_maupiti = W_maupiti/W_maupiti.sum(axis=1)[:,None]

In [23]:
#X_maupiti_2 = np.ones((n_samples,1+n_features*2))
#X_maupiti_2[:,1:1+n_features] = X_maupiti
#X_maupiti_2[:,1+n_features:] = X_maupiti**2

X_maupiti_1 = np.ones((n_samples,1+n_features))
X_maupiti_1[:,1:] = X_maupiti

In [24]:
Y_star_maupiti = (Y_maupiti*(n_samples-1)+1/n_classes)/n_samples

In [25]:
beta_maupiti = np.zeros((1+n_features)*n_classes+(1+n_features))[:(1+n_features)*n_classes].reshape(((1+n_features),n_classes))
beta_maupiti[:,0] = 0

In [26]:
gamma_maupiti = np.zeros(Z_maupiti.shape[-1])
rho_maupiti = [0.]

In [27]:
params0_maupiti = np.concatenate([beta_maupiti.flatten(),gamma_maupiti,rho_maupiti])

In [50]:
min_bounds = -np.inf*np.ones(len(params0_maupiti)) 
max_bounds = np.inf*np.ones(len(params0_maupiti))
min_bounds[-1] = -1
max_bounds[-1] = 1
bounds = Bounds(min_bounds, max_bounds)

In [99]:
X_maupiti_train = X_maupiti_1[1000:1500,:]
Y_star_maupiti_train = Y_star_maupiti[1000:1500,:]
Z_maupiti_train = Z_maupiti[1000:1500]
W_maupiti_train = W_maupiti[1000:1500,1000:1500]

In [109]:
%%time
solution_maupiti = minimize(f_spatial, params0_maupiti, args=(X_maupiti_train, 
                                                             Y_star_maupiti_train, 
                                                             Z_maupiti_train, W_maupiti_train),
                            jac=fprime_spatial, bounds=bounds)

Wall time: 4.71 s


In [110]:
solution_maupiti.x

array([ 0.00000000e+00, -1.15892463e-01, -1.86093568e-01, -1.74398104e-01,
       -1.75067441e-01, -1.74398104e-01,  8.40833285e-03,  7.69632506e-02,
       -1.71796593e-01, -1.61087239e-01,  0.00000000e+00, -2.99999232e-03,
       -5.75462589e-05, -4.70997713e-04, -4.32577842e-04, -4.70997713e-04,
        7.89220564e-03,  4.88999652e-03, -3.42719855e-04, -5.21630993e-04,
        0.00000000e+00,  4.03341826e-01,  5.41092222e-01, -1.34828736e-01,
       -9.17238845e-02, -1.34828736e-01,  5.75872925e-01, -7.39470955e-01,
       -1.45802268e-01, -1.65397158e-01,  0.00000000e+00,  2.52370911e-06,
        6.19833402e-07,  4.15285716e-08,  6.59586534e-08,  4.15285716e-08,
       -2.26830237e-06, -1.75606779e-06, -1.63945391e-08, -1.48231826e-07,
        0.00000000e+00,  8.32706451e-06,  6.20823911e-07, -5.01676442e-07,
       -5.04785148e-07, -5.01676442e-07,  5.87148610e-07, -1.20094799e-06,
       -5.93255283e-07,  6.29925284e-07,  0.00000000e+00, -1.03378528e-03,
       -1.91488182e-05, -

In [111]:
beta_solution = solution_maupiti.x[:n_classes*(n_features+1)].reshape((1+n_features,n_classes))

In [112]:
rho_solution = solution_maupiti.x[-1]

In [113]:
mu_maupiti = dirichlet_regression.compute_mu_spatial_2(X_maupiti_train, beta_solution, rho_solution, W_maupiti_train)

In [114]:
r2_score(Y_star_maupiti_train, mu_maupiti)

-5.970576435203907e+34

In [130]:
mu_maupiti_pred = dirichlet_regression.compute_mu_spatial(X_maupiti_1, beta_solution, rho_solution, W_maupiti)
r2_score(Y_star_maupiti, mu_maupiti_pred)

-0.24609029796843368

In [116]:
mean_squared_error(Y_star_maupiti, mu_maupiti_pred, squared=False)

0.2446366085729225

### With whole dataset

In [151]:
#params0_bis = np.copy(solution_maupiti.x)

In [162]:
%%time
solution_maupiti_bis = minimize(f_spatial, params0_maupiti, args=(X_maupiti_1, 
                                                             Y_star_maupiti, 
                                                             Z_maupiti, W_maupiti), 
                                jac=fprime_spatial, bounds=bounds)

C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:59: RuntimeWarning: overflow encountered in exp
  mu = np.zeros((n,J))
C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:61: RuntimeWarning: invalid value encountered in true_divide
  sum_exp_MXbeta = np.sum(exp_MXbeta,axis=1)
C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:105: RuntimeWarning: invalid value encountered in multiply
  digamma_phi_mu = digamma(phi[i]*mu[i])
C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:119: RuntimeWarning: invalid value encountered in multiply
  for i in range(n):
C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:268: RuntimeWarning: invalid value encountered in multiply
  digamma_alpha = digamma(alpha[i])
<ipython-input-107-5714c1b972bd>:13: RuntimeWarning: overflow encountered in exp
  phi = np.exp(np.matmul(Z,gamma_var))
C:\Users\tnguyen001\Documents\GitHub\maupiti\dirichlet_regression.py:90: RuntimeWar

Wall time: 7min 57s


In [163]:
beta_solution_bis = solution_maupiti.x[:n_classes*(n_features+1)].reshape((1+n_features,n_classes))
rho_solution_bis = solution_maupiti.x[-1]
mu_maupiti_bis = dirichlet_regression.compute_mu_spatial(X_maupiti_1, beta_solution_bis, rho_solution_bis, W_maupiti)

In [164]:
print('R2 :',r2_score(Y_star_maupiti, mu_maupiti_bis))
print('RMSE :',mean_squared_error(Y_star_maupiti, mu_maupiti_bis, squared=False))

R2 : -0.24609029796843368
RMSE : 0.2449686094095783


In [165]:
print(rho_solution_bis)

0.41541644652962506


## With X^2

In [28]:
X_arctic_2 = np.ones((39,3))
X_arctic_2[:,1] = X_arctic[:,0]
X_arctic_2[:,2] = X_arctic[:,0]**2

# Using the derivatives

In [465]:
def fhessian(x, X, Y, Z):
    K = X.shape[-1]
    J = Y.shape[-1]
    beta = x[:K*J].reshape((K,J))
    beta[:,0] = 0
    gamma_var = x[K*J:]
    #mu = dirichlet_regression.compute_mu(X, beta)
    mu = dirichlet_regression.compute_mu_2(X, beta)
    phi = np.exp(np.matmul(Z,gamma_var))
    if len(phi.shape)==1:
        K_phi=1
    else:
        K_phi = phi.shape[-1]
    hessian = np.zeros((K*J+1,K*J+1))
    #beta_hess = dirichlet_regression.dirichlet_hessian_wrt_beta(mu, phi, X, Y).reshape((K*J,K*J))
    beta_hess = dirichlet_hessian_wrt_beta_2(mu, phi, X, Y).reshape((K*J,K*J))
    #gamma_hess = dirichlet_regression.dirichlet_hessian_wrt_gamma(mu, phi, beta, X, Y, Z)
    gamma_hess = dirichlet_regression.dirichlet_hessian_wrt_gamma_2(mu, phi, beta, X, Y, Z)
    #beta_gamma_derivative = dirichlet_regression.dirichlet_second_derivative_beta_gamma(mu, phi, beta, X, Y, Z).reshape((K_phi,K*J))
    beta_gamma_derivative = dirichlet_regression.dirichlet_second_derivative_beta_gamma_2(mu, phi, beta, X, Y, Z).reshape((K_phi,K*J))
    
    # we set to 0 the first columns of the betas
    for k in range(0,K*J,J):
        beta_hess[k,:] = 0
        beta_hess[:,k] = 0.
        beta_gamma_derivative[:,k] = 0.
    hessian[:K*J,:K*J] = beta_hess
    hessian[K*J:,K*J:] = gamma_hess
    hessian[:K*J,K*J:] = beta_gamma_derivative.transpose()
    hessian[K*J:,:K*J] = beta_gamma_derivative
    return(-hessian)

In [113]:
def trigamma_func(z):
    S=0
    for n in range(10000):
        S+=1/(z+n)**2
    return S

def z_times_trigamma_func(z):
    S=0
    for n in range(10000):
        S+=1/( z + 2*n + (n**2)/z )
    return S

def a_times_trigamma_func(a,z):
    S=0
    try:
        for n in range(10000):
            S+=1/( z*(z/a) + (2*z*n)/a + (n**2)/a )
    except ZeroDivisionError:
        S=1e305
    return S

In [236]:
def dirichlet_hessian_wrt_beta_2(mu, phi, X, Y, alpha=None, epsilon=0):
    K = np.shape(X)[1] #nb of features
    J = np.shape(Y)[1] #nb of classes
    n = np.shape(Y)[0] 
    hessian = np.zeros((K,J,K,J))
    if alpha is None:
        alpha = np.copy(mu)
        for j in range(J):
            alpha[:,j] = phi*mu[:,j]
    for i in range(n):
        digamma_alpha = digamma(alpha[i])
        digamma_alpha[np.isinf(digamma_alpha)] = -1e305

        trigamma_alpha = polygamma(1,alpha[i])
        trigamma_alpha[np.isinf(trigamma_alpha)] = 1e305
        
        #mu_2_trigamma = a_times_trigamma_func(mu[i]**2,alpha[i])
        #if math.isinf(trigamma_alpha):
        #    mu_2_trigamma = a_times_trigamma_func(mu[i]**2,alpha[i])
        #else:
        mu_2_trigamma = mu[i]**2*trigamma_alpha
        mu_2_trigamma[mu[i]**2==0] = 0
        phi_sum_mu_2_trigamma = phi[i]*np.sum(mu_2_trigamma)
        
        logY = np.log(Y[i]+epsilon)
        sum_mu_log = 2*np.sum(mu[i]*logY)
        
        mu_digamma_alpha = mu[i]*digamma_alpha
        mu_digamma_alpha[mu[i]==0] = 0
        sum_mu_digamma = 2*np.sum(mu_digamma_alpha)
        
        mu_digamma_log = mu[i]*(digamma_alpha-logY)
        #mu_digamma_log[mu[i]==0] = 0
        sum_mu_digamma_log = np.sum(mu_digamma_log)
        for p in range(K):
            phi_Xip = phi[i]*X[i,p]
            for d in range(J):
                phi_Xip_muid = phi_Xip*mu[i,d]
                if alpha[i,d]==0:
                    alpha_times_trigamma_alpha = np.zeros(np.shape(trigamma_alpha[d]))
                else:
                    if math.isinf(trigamma_alpha[d]):
                        alpha_times_trigamma_alpha = np.array(a_times_trigamma_func(alpha[i,d],alpha[i,d]))
                    else:
                        alpha_times_trigamma_alpha = alpha[i,d]*trigamma_alpha[d]
                for q in range(K):
                    phi_Xip_muid_Xiq = phi_Xip_muid*X[i,q]
                    for c in range(J):
                        if c!=d:
                            if alpha[i,c]==0:
                                alpha_c_trigamma_alpha_c = 0
                            elif math.isinf(trigamma_alpha[c]):
                                alpha_c_trigamma_alpha_c = a_times_trigamma_func(alpha[i,c],alpha[i,c])
                            else:
                                alpha_c_trigamma_alpha_c = alpha[i,c]*trigamma_alpha[c]
                            hessian[p,d,q,c] += phi_Xip_muid_Xiq*mu[i,c] * ( digamma_alpha[c] + digamma_alpha[d]
                                                                           - logY[c] - logY[d]
                                                                           + alpha_c_trigamma_alpha_c + alpha_times_trigamma_alpha
                                                                           - phi_sum_mu_2_trigamma 
                                                                           + sum_mu_log
                                                                           - sum_mu_digamma )
                        else:
                            
                            if math.isinf(trigamma_alpha[c]):
                                #phi_trigamma_alpha_c = np.array(a_times_trigamma_func(phi[i],alpha[i,c]))
                                phi_trigamma_alpha_c = phi[i]*1e305
                            else:
                                phi_trigamma_alpha_c = phi[i]*trigamma_alpha[c]
                            hessian[p,d,q,c] += phi_Xip_muid_Xiq*mu[i,c] * ( 2*digamma_alpha[c] - 2*logY[c] 
                                                                           + 2*alpha_times_trigamma_alpha
                                                                           - phi_sum_mu_2_trigamma
                                                                           + sum_mu_log
                                                                           - sum_mu_digamma
                                                                           - phi_trigamma_alpha_c)
                            hessian[p,d,q,c] += phi_Xip_muid_Xiq * (sum_mu_digamma_log - digamma_alpha[c] + logY[c] )
    # fill the symmetric values we didn't compute
    #for p in range(K):
    #    for d in range(J):
    #        for q in range(p):
    #            for c in range(J):
    #                hessian[p,d,q,c] = hessian[q,c,p,d]
    return hessian